In [1]:
import numpy as np
from numpy import expand_dims as ed
%load_ext autoreload
%autoreload 2

In [53]:
a = np.ones((4,4,14))
b=np.ones((2,6,6))

In [59]:
c = np.zeros((2,3,3,14))

In [12]:
a

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [13]:
b

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [17]:
c = a[None,:,:]*b

In [18]:
c.shape

(2, 4, 4)

In [27]:
c.sum(axis=1).sum(axis=1)

array([0., 0.])

In [29]:
a[None,:,:]

array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]])

In [54]:
(ed(a,0)*b[:,2:,2:,None]).shape

(2, 4, 4, 14)

In [66]:
(ed(a,0)*b[:,2:,2:,None]).sum(axis=1).sum(axis=1).shape

(2, 14)

In [68]:
c[:,2,2] = (ed(a,0)*b[:,2:,2:,None]).sum(axis=1).sum(axis=1)

In [69]:
c

array([[[[16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
          16., 16.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.],
         [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
          16., 16.]]],


       [[[16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
          16., 16.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0

In [70]:
sqrt = np.sqrt(np.arange(4))
sqrtT = sqrt.reshape(-1, 1)

In [72]:
a[:,:-1]*sqrt[1:,None]

array([[[1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        ],
        [1.41421356, 1.41421356, 1.41421356, 1.41421356, 1.41421356,
         1.41421356, 1.41421356, 1.41421356, 1.41421356, 1.41421356,
         1.41421356, 1.41421356, 1.41421356, 1.41421356],
        [1.73205081, 1.73205081, 1.73205081, 1.73205081, 1.73205081,
         1.73205081, 1.73205081, 1.73205081, 1.73205081, 1.73205081,
         1.73205081, 1.73205081, 1.73205081, 1.73205081]],

       [[1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        ],
        [1.41421356, 1.41421356, 1.41421356, 1.41421356, 1.41421356,
         1.41421356, 1.41421356, 1.41421356, 1.41421356, 1.41421356,
         1.41421356, 1.41421356, 1.41421356, 1.41421356],
        [1.7320508

In [77]:
ed(sqrtT[1:],1).shape


(3, 1, 1)

In [90]:
a.shape

(4, 4, 14)

In [83]:
ed(ed(a,0),2).shape

(1, 4, 1, 4, 14)

In [84]:
A = 3

In [91]:
ed(ed(ed(a,0),1),-1).shape

(1, 1, 4, 4, 14, 1)

In [437]:

def R_matrix2(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi, old_state):
    batch, cutoff, _ = old_state.shape
    dtype = old_state.dtype
    
    C, mu ,Sigma = C_mu_Sigma2(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi)

    sqrt = np.sqrt(np.arange(cutoff, dtype = dtype))
    sqrtT = sqrt.reshape(-1, 1)

    R = np.zeros((batch, cutoff, cutoff, cutoff+1, cutoff+1), dtype = dtype)
    G_00pq = np.zeros((cutoff, cutoff+1), dtype = dtype)
    
    
    #G_mn00
    G_00pq[0,0] = C
    for q in range(1, cutoff):
        G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]

    
    for p in range(1,cutoff):
        for q in range(0,cutoff):
            G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]
              
#     for p in range(1,cutoff):
#         G_00pq[p,:-1] = (mu[2]*G_00pq[p-1,:-1] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,:-1] - Sigma[2,3]*G_00pq[p-1,1:])/sqrt[p]


    # R_00^jk = a_dagger^j \G_00pq> b^k  * |old_state>
    G_00pq2 = G_00pq[:,:-1]
    for j in range(cutoff):
        G_00pq3 = G_00pq2
        for k in range(cutoff):
            R[0,0,j,k] = np.sum(G_00pq3*old_state[:,j:,k:]) #!!here only works for batch = 1, if not the np.sum needs to be changed axis=-1 for several times = batch.
            G_00pq3 = G_00pq3[:,:-1]*sqrt[k+1:]
        G_00pq2 = sqrtT[j+1:]*G_00pq2[:-1,:]

    #R_0n^jk
    
#     for n in range(1,cutoff):
#         for k in range(0,cutoff):
#             for j in range(0,cutoff):
#                 R[:,0,n,j,k] = mu[1]/sqrt[n]*R[:,0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[:,0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[:,0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[:,0,n-1,j,k+1]
   
    for n in range(1,cutoff):
        R[:,0,n,:-1,:-1] = mu[1]/sqrt[n]*R[:,0,n-1,:-1,:-1] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[:,0,n-2,:-1,:-1] - Sigma[1,2]/sqrt[n]*R[:,0,n-1,1:,:-1] - Sigma[1,3]/sqrt[n]*R[:,0,n-1,:-1,1:]


    #R_mn^jk
    for m in range(1,cutoff):
        for n in range(0,cutoff):
            for j in range(0,cutoff-m):
                for k in range(0,cutoff-m-j):
                    R[:,m,n,j,k] = mu[0]/sqrt[m]*R[:,m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[:,m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[:,m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[:,m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[:,m-1,n,j,k+1]   
    return R

In [438]:
old_state = np.ones((1,2,2),dtype=np.complex128)

In [439]:
gamma1 = 1+1j
gamma2= 1+1j
phi1 =0.1
phi2 =0.2
theta1 = 0.4
varphi1 = 0.4
zeta1= 1+1j
zeta2= 1+1j
theta=0.6
varphi=0.2

In [440]:
R1 = R_matrix2(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi, old_state)

In [441]:
def R_matrix22(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi, old_state):
    """
    Directly constructs the transformed state recursively and exactly.

    Arguments:
        gamma1 (complex): displacement parameter1
        gamma2 (complex): displacement parameter2
        phi1 (float): phase rotation parameter1
        phi2 (float): phase rotation parameter2
        
        theta1(float): transmissivity angle of the beamsplitter1
        varphi1(float): reflection phase of the beamsplitter1
        
        zeta1 (complex): squeezing parameter1
        zeta2 (complex): squeezing parameter2
        
        theta(float): transmissivity angle of the beamsplitter
        varphi(float): reflection phase of the beamsplitter
        
        old_state(array([batch,D,D])): State to be transformed

    Returns:
        R (complex array[batch,D,D,D,D]): the matrix where R[batch,:,:,0,0] is the transformed state for each batch
    """
    batch, cutoff, _ = old_state.shape
    dtype = old_state.dtype

    
    C, mu ,Sigma = C_mu_Sigma2(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi)

    sqrt = np.sqrt(np.arange(cutoff, dtype = dtype))
    sqrtT = sqrt.reshape(-1, 1)

    R = np.zeros((batch, cutoff, cutoff, cutoff+1, cutoff+1), dtype = dtype)
    G_00pq = np.zeros((cutoff, cutoff), dtype = dtype)
    
    
    #G_mn00
    G_00pq[0,0] = C
    for q in range(1, cutoff):
        G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]


    for p in range(1,cutoff):
        for q in range(0,cutoff):
            G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]
        
    # R_00^jk = a_dagger^j \G_00pq> b^k  * |old_state>
    G_00pq2 = G_00pq
    for j in range(cutoff):
        G_00pq3 = G_00pq2
        for k in range(cutoff):
            R[0,0,j,k] = np.sum(G_00pq3*old_state[:,j:,k:]) #!!here only works for batch = 1, if not the np.sum needs to be changed axis=-1 for several times = batch.
            G_00pq3 = G_00pq3[:,:-1]*sqrt[k+1:]
        G_00pq2 = sqrtT[j+1:]*G_00pq2[:-1,:]

    #R_0n^jk
    for n in range(1,cutoff):
        for k in range(0,cutoff):
            for j in range(0,cutoff):
                R[:,0,n,j,k] = mu[1]/sqrt[n]*R[:,0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[:,0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[:,0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[:,0,n-1,j,k+1]


    #R_mn^jk
    for m in range(1,cutoff):
        for n in range(0,cutoff):
            for j in range(0,cutoff-m):
                for k in range(0,cutoff-m-j):
                    R[:,m,n,j,k] = mu[0]/sqrt[m]*R[:,m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[:,m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[:,m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[:,m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[:,m-1,n,j,k+1]
          
    return R

In [442]:
R2 = R_matrix22(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi, old_state)

In [443]:
from poenta.jitted import C_mu_Sigma2,dC_dmu_dSigma2

In [467]:
from numba import njit
@njit()
def dPsi2(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi, state_in, G00, R):
    """
    Computes the gradient of the new state with respect to
    gamma, gamma*, phi, z, z* but not with respect to the old state

    Arguments:
        gamma1 (complex): displacement parameter1
        gamma2 (complex): displacement parameter2
        phi1 (float): phase rotation parameter1
        phi2 (float): phase rotation parameter2
        
        theta1(float): transmissivity angle of the beamsplitter1
        varphi1(float): reflection phase of the beamsplitter1
        
        zeta1 (complex): squeezing parameter1
        zeta2 (complex): squeezing parameter2
        
        theta(float): transmissivity angle of the beamsplitter
        varphi(float): reflection phase of the beamsplitter
        
        state_in: (complex array[bath,D,D]): old state
        G00 (complex array[D,D]): G[0,0,:,:] of the G matrix
        R (complex array[bath, D,D,D,D]): complete R matrix R[:,:,:,:] (!not really complete....)

    Returns:
        (complex array[batch, D, D, 14]): gradient of the new state with respect to
                                    gamma1, gamma1*, gamma2, gamma2*, phi1, phi2, theta1, varphi1, zeta1, zeta1*, zeta2, zeta2*, theta, varphi
    """
    batch, cutoff, _ = state_in.shape
    dtype = state_in.dtype
    
    C, mu, Sigma = C_mu_Sigma2(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi)
    dC, dmu, dSigma = dC_dmu_dSigma2(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi)
    
    sqrt = np.sqrt(np.arange(cutoff, dtype=dtype))
    sqrtT = sqrt.reshape(-1, 1)
    
    dR = np.zeros((batch, cutoff, cutoff, cutoff+1 , cutoff+1, 14),dtype = dtype)
    dG00 = np.zeros((cutoff, cutoff, 14),dtype = dtype)
    
    dG00[0,0] = dC
    for q in range(1, cutoff):
        dG00[0,q] = (dmu[3]*G00[0,q-1]+mu[3]*dG00[0,q-1] - dSigma[3,3]*sqrt[q-1]*G00[0,q-2]- Sigma[3,3]*sqrt[q-1]*dG00[0,q-2])/sqrt[q]


    for p in range(1,cutoff):
        for q in range(0,cutoff):
            dG00[p,q] = (dmu[2]*G00[p-1,q]+ mu[2]*dG00[p-1,q] - dSigma[2,2]*sqrt[p-1]*G00[p-2,q]- Sigma[2,2]*sqrt[p-1]*dG00[p-2,q] - dSigma[2,3]*sqrt[q]*G00[p-1,q-1]- Sigma[2,3]*sqrt[q]*dG00[p-1,q-1])/sqrt[p]
                    
    dG002 = dG00
    for j in range(cutoff):
        dG003 = dG002
        for k in range(cutoff):
        #dG003[D,D,14]*state_in[batch,D,D] - > we want [batch,14]
            test = ed(dG003,0)*ed(state_in[:,j:,k:],-1)
            dR[:,0,0,j,k] = test.sum(axis=1).sum(axis=1)
            dG003 = dG003[:,:-1]*ed(sqrt[k+1:],1)
        dG002 = ed(sqrtT[j+1:],1)*dG002[:-1,:]

    for n in range(1,cutoff):
        for k in range(0,cutoff):
            for j in range(0,cutoff):
            #dR[batch,D,D,D,D,14] R[batch,D,D,D,D] dmu[4,14] dSigma[4,4,14]
            # dR[:,0,n,j,k] = [batch,14]
#                 dR[:,0,n,j,k] = (dmu[1]*R[:,0,n-1,j,k] + ed(mu[1],0)*dR[:,0,n-1,j,k] - dSigma[1,1]*sqrt[n-1]*R[:,0,n-2,j,k] - Sigma[1,1]*sqrt[n-1]*dR[:,0,n-2,j,k] - dSigma[1,2]*R[:,0,n-1,j+1,k] - Sigma[1,2]*dR[:,0,n-1,j+1,k] - dSigma[1,3]*R[:,0,n-1,j,k+1] - Sigma[1,3]*dR[:,0,n-1,j,k+1]
#                                 )/sqrt[n]
                dR[:,0,n,j,k] = (ed(dmu[1],0)*R[:,0,n-1,j,k]+ mu[1]*dR[:,0,n-1,j,k]- ed(dSigma[1,1],0)*sqrt[n-1]*R[:,0,n-2,j,k]- Sigma[1,1]*sqrt[n-1]*dR[:,0,n-2,j,k] - ed(dSigma[1,2],0)*R[:,0,n-1,j+1,k] - Sigma[1,2]*dR[:,0,n-1,j+1,k] - ed(dSigma[1,3],0)*R[:,0,n-1,j,k+1] - Sigma[1,3]*dR[:,0,n-1,j,k+1])/sqrt[n]


    for m in range(1,cutoff):
        for n in range(0,cutoff):
            for j in range(0,cutoff-m):
                for k in range(0,cutoff-m-j):
#                     dR[:,m,n,j,k] = (dmu[0]*R[m-1,n,j,k] + mu[0]*dR[:,m-1,n,j,k] - dSigma[0,0]*sqrt[m-1]*R[m-2,n,j,k] - Sigma[0,0]*sqrt[m-1]*dR[:,m-2,n,j,k] - dSigma[0,1]*sqrt[n]*R[m-1,n-1,j,k] - Sigma[0,1]*sqrt[n]*dR[:,m-1,n-1,j,k] - dSigma[0,2]*R[m-1,n,j+1,k] - Sigma[0,2]*dR[:,m-1,n,j+1,k] - dSigma[0,3]*R[m-1,n,j,k+1] - Sigma[0,3]*dR[:,m-1,n,j,k+1])/sqrt[m]
                    dR[:,m,n,j,k] = (ed(dmu[0],0)*R[:,m-1,n,j,k] + mu[0]*dR[:,m-1,n,j,k]  - ed(dSigma[0,0],0)*sqrt[m-1]*R[:,m-2,n,j,k]  - Sigma[0,0]*sqrt[m-1]*dR[:,m-2,n,j,k]- ed(dSigma[0,1],0)*sqrt[n]*R[:,m-1,n-1,j,k] - Sigma[0,1]*sqrt[n]*dR[:,m-1,n-1,j,k] - ed(dSigma[0,2],0)*R[:,m-1,n,j+1,k] - Sigma[0,2]*dR[:,m-1,n,j+1,k]  - ed(dSigma[0,3],0)*R[:,m-1,n,j,k+1]  - Sigma[0,3]*dR[:,m-1,n,j,k+1])/sqrt[m]
    return np.transpose(dR[:,:,:,0,0,:],(3,0,1,2))

In [468]:
gamma1 = 1+1j
gamma2= 1+1j
phi1 =0.1
phi2 =0.2
theta1 = 0.4
varphi1 = 0.4
zeta1= 1+1j
zeta2= 1+1j
theta=0.6
varphi=0.2
state_in = np.ones((1,2,2),dtype=np.complex128)
G00 = np.ones((2,2),dtype=np.complex128)
R=R1

In [469]:
dpsi2 = dPsi2(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi, state_in, G00, R)

In [470]:
dpsi2.shape

(14, 1, 2, 2)

In [365]:
dR[:,0,n,j,k] = 
(ed(dmu[1],0)*R[:,0,n-1,j,k]
 + ed(mu[1],0)*dR[:,0,n-1,j,k]
 - ed(dSigma[1,1],0)*sqrt[n-1]*R[:,0,n-2,j,k]
 - Sigma[1,1]*sqrt[n-1]*dR[:,0,n-2,j,k] 
 - ed(dSigma[1,2],0)*R[:,0,n-1,j+1,k] 
 - Sigma[1,2]*dR[:,0,n-1,j+1,k] 
 - ed(dSigma[1,3],0)*R[:,0,n-1,j,k+1] 
 - Sigma[1,3]*dR[:,0,n-1,j,k+1])/sqrt[n]



In [450]:
dR = np.zeros((1, 2, 2, 3 , 3, 14),dtype = np.complex128)

In [462]:
#dR[batch,D,D,D,D,14] R[bath,D,D,D,D] dmu[4,14] dSigma[4,4,14]
# dR[:,0,n,j,k] = [batch,14]
j=0
k=0
n=0
m=1
dR[:,m,n,j,k] = (ed(dmu[0],0)*R[:,m-1,n,j,k] 
                 + mu[0]*dR[:,m-1,n,j,k] 
                 - ed(dSigma[0,0],0)*sqrt[m-1]*R[:,m-2,n,j,k] 
                 - Sigma[0,0]*sqrt[m-1]*dR[:,m-2,n,j,k]
                 - ed(dSigma[0,1],0)*sqrt[n]*R[:,m-1,n-1,j,k] 
                 - Sigma[0,1]*sqrt[n]*dR[:,m-1,n-1,j,k] 
                 - ed(dSigma[0,2],0)*R[:,m-1,n,j+1,k] 
                 - Sigma[0,2]*dR[:,m-1,n,j+1,k] 
                 - ed(dSigma[0,3],0)*R[:,m-1,n,j,k+1] 
                 - Sigma[0,3]*dR[:,m-1,n,j,k+1]
                )/sqrt[m]
           

In [463]:
dC, dmu, dSigma = dC_dmu_dSigma2(gamma1, gamma2, phi1, phi2, theta1, varphi1, zeta1, zeta2, theta, varphi)
    

In [464]:
ed(dmu[0],0).shape

(1, 14)

In [465]:
R[:,0,0,0,0].shape

(1,)

In [466]:
ed(ed(dSigma[0,0],0),0)*sqrt[m-1]*ed(R[:,m-2,n,j,k],-1)

array([[[-0.+0.j, -0.+0.j, -0.+0.j, -0.+0.j,  0.-0.j, -0.+0.j, -0.+0.j,
         -0.+0.j, -0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j]]])

In [455]:
ed(dSigma[0,0],0)*sqrt[m-1]*R[:,m-2,n,j,k]

array([[-0.+0.j, -0.+0.j, -0.+0.j, -0.+0.j,  0.-0.j, -0.+0.j, -0.+0.j,
        -0.+0.j, -0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j]])

In [456]:
 ed(mu[0],0)*dR[:,m-1,n,j,k] 

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [391]:
dR[:,0,0,0,0] 

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [312]:
R[:,0,0,0,1].shape

(1,)

In [324]:
(ed(dSigma[1,1],0)*R[:,0,0,0,0]).shape

(1, 14)

In [316]:
ed(dmu[1],0)*R[:,0,0,0,0]

array([[ 0.        +0.j        , -0.00128154-0.00109039j,
         0.00251752+0.00629543j, -0.00459624+0.00351062j,
        -0.00019115-0.00237193j, -0.01640487-0.00454318j,
        -0.0089537 -0.00156751j, -0.00737348-0.0014361j ,
         0.00062713+0.00189856j,  0.00094461-0.00031202j,
         0.00098901+0.0020264j ,  0.00100821-0.00049207j,
         0.        +0.j        ,  0.        +0.j        ]])

In [326]:
dR[:,0,0,0,0].shape

(1, 14)